In [ ]:
#주석풀고 설치해주면 colab에서 geopandas가 돌아갑니다!

import requests
import zipfile


def save_file(url, file_name):
  r = requests.get(url)
  with open(file_name, 'wb') as f:
    f.write(r.content)

    
save_file('https://courses.cs.washington.edu/courses/cse163/19sp/' +
          'files/lectures/05-13/data.zip', 'data.zip')
save_file('https://courses.cs.washington.edu/courses/cse163/19sp/' +
          'files/lectures/05-13/gz_2010_us_040_00_5m.json', 
          'gz_2010_us_040_00_5m.json')
save_file('https://courses.cs.washington.edu/courses/cse163/19sp/' +
          'files/lectures/05-13/stormhistory.csv', 'stormhistory.csv')

with zipfile.ZipFile("data.zip","r") as zip_ref:
    zip_ref.extractall()
    
!pip install --upgrade geopandas
!pip install --upgrade pyshp
!pip install --upgrade shapely
!pip install --upgrade descartes

In [2]:
import pathlib
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, Point
import descartes

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/My\ Drive/compas2020/data

/content/drive/My Drive/compas2020/data


In [18]:
final_parkinglot = pd.read_csv('광양시_주차장_공간정보_전처리.csv')
final_parkinglot

,Unnamed: 0,주차장명칭,구분,DGM_AR,주차면수,유료/무료,주소,lon,lat,geometry,category,소유주체
0,0,인동숲앞 주차장,노외,4189.0,161.0,유료,인서리 237-1,127.586883,34.970324,POINT (127.5868828 34.97032367),주차장,군유지
1,1,구터미널주차장,노외,2968.0,125.0,유료,중동 1651,127.695428,34.936252,POINT (127.6954279 34.93625189999999),주차장,군유지
2,2,구경찰서 주차장,노외,2836.0,89.0,무료,읍내리 227-1,127.584586,34.975637,POINT (127.5845858 34.97563677),주차장,군유지
3,3,칠성1지구,노외,1980.0,52.0,무료,칠성리 962-1,127.580683,34.970130,POINT (127.5806828 34.97013011),주차장,군유지
4,4,칠성1지구,노외,1970.0,64.0,무료,칠성리 948-1,127.580707,34.972523,POINT (127.5807067 34.97252282),주차장,군유지
...,...,...,...,...,...,...,...,...,...,...,...,...
230,230,포석정위도로변,피서지,264.0,10.0,무료,동곡리 662-1,127.604822,35.079214,POINT (127.6048219 35.07921402),주차장,군유지
231,231,동곡산장옆,피서지,1212.0,60.0,무료,동곡리 549,127.613417,35.070673,POINT (127.6134168 35.07067277),주차장,군유지
232,232,하천첨주변,피서지,2381.0,96.0,무료,산남리 154-4,127.611774,35.002759,POINT (127.6117741 35.00275939),주차장,개인
233,233,회계분교,피서지,2090.0,66.0,무료,진상 어치 1140,127.674719,35.092388,POINT (127.6747186 35.09238804),주차장,군유지


In [179]:
a = final_parkinglot['DGM_AR']
s = set()
for i in a:
  if str(i) == "nan":
    pass
  else:
    z = len(str(int(i)))
    s.add(z)
s

{2, 3, 4, 5}

In [180]:
head =[]

#index 0~3까지는 각 면적의 자릿수+ 앞자리 별 개수, 4~7까지는 각 면적의 자릿수+ 앞자리 계신 값 들의 sum 
for i in range(len(s)+4):
  head.append([0,0,0,0,0,0,0,0,0,0])
print(head)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [62]:
b = final_parkinglot['주차면수']

# n = 면적//주차면수 
def count_parkinglot(x, y):
  n = (x//y)
  return n

def head_index(num1, num2, x, y):
  #각각의 n값과, 개수 추가
  for i in range(10):
    if int(x) ==i:
      head[num1][i]+=1
      head[num2][i] +=y

count =0 
for i in a:
  if str(i) == "nan" or str(b[count])== "nan":
    pass
  else:
    if len(str(int(i))) == 2:
      num=count_parkinglot(i, b[count])
      head_index(0, 4, str(int(i))[0], num)
    elif len(str(int(i))) == 3:
      num=count_parkinglot(i, b[count])
      head_index(1, 5, str(int(i))[0], num)
    elif len(str(int(i))) == 4:
      num=count_parkinglot(i, b[count])
      head_index(2, 6, str(int(i))[0], num)
    elif len(str(int(i))) == 5:
      num=count_parkinglot(i, b[count])
      head_index(3, 7, str(int(i))[0], num)
  count+=1


In [63]:
head

[[0, 0, 0, 0, 0, 0, 0, 2, 3, 0],
 [0, 11, 36, 15, 14, 14, 7, 10, 8, 6],
 [0, 40, 15, 10, 6, 2, 3, 0, 0, 0],
 [0, 0, 1, 2, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 32.0, 30.0, 0],
 [0, 260.0, 861.0, 394.0, 421.0, 406.0, 198.0, 322.0, 213.0, 192.0],
 [0, 1198.0, 456.0, 368.0, 236.0, 59.0, 158.0, 0, 0, 0],
 [0, 0, 46.0, 2598.0, 0, 0, 0, 0, 0, 0]]

In [87]:
_count =0
_count1=3

head2 =[]

for i in range(4):
  head2.append([0,0,0,0,0,0,0,0,0,0])

for i in head[4:8]:
  for j in range(10):
    if head[_count][j]!=0:
      head2[_count][j]=i[j]//head[_count][j]
  _count+=1
  _count1+=1
  
head2

[[0, 0, 0, 0, 0, 0, 0, 16.0, 10.0, 0],
 [0, 23.0, 23.0, 26.0, 30.0, 29.0, 28.0, 32.0, 26.0, 32.0],
 [0, 29.0, 30.0, 36.0, 39.0, 29.0, 52.0, 0, 0, 0],
 [0, 0, 46.0, 1299.0, 0, 0, 0, 0, 0, 0]]

In [88]:
plan_parkinglot = pd.read_csv('광양시_도시계획(교통시설)_전처리.csv')
plan_parkinglot

,PRESENT_SN,DGM_NM,DGM_AR,DGM_LT,geometry,lon,lat,category,소유주체
0,46230UQ152PS201202010003,기타 주차장시설,344,174,POINT (127.5842466468615 34.97703873009804),127.584247,34.977039,주차장,군유지
1,46230UQ152PS201202010002,기타 주차장시설,483,94,POINT (127.5838975833261 34.9783680381622),127.583898,34.978368,주차장,군유지
2,46230UQ152PS201202010109,노외주차장,1612,167,POINT (127.7513775866315 34.9457556003597),127.751378,34.945756,주차장,NaN
3,46230UQ152PS201202010106,노외주차장,305,70,POINT (127.7027049702007 34.98893978682275),127.702705,34.988940,주차장,군유지
4,46230UQ152PS201202010105,기타 주차장시설,4608,286,POINT (127.6896377942097 34.93744594755481),127.689638,34.937446,주차장,군유지
...,...,...,...,...,...,...,...,...,...
203,46230UQ152PS201202010153,노외주차장,805,127,POINT (127.5899412601061 34.97387051256062),127.589941,34.973871,주차장,개인
204,46230UQ152PS201202010143,기타 주차장시설,1244,144,POINT (127.6665902516217 34.93273694287271),127.666590,34.932737,주차장,NaN
205,46230UQ152PS201202010139,노외주차장,2657,208,POINT (127.5846080438416 34.97562896258442),127.584608,34.975629,주차장,군유지
206,46230UQ152PS201202010130,노외주차장,1313,142,POINT (127.6984953327121 34.92671326177527),127.698495,34.926713,주차장,군유지


In [89]:
pp = plan_parkinglot['DGM_AR']
s = set()
for i in pp:
  if str(i) == "nan":
    pass
  else:
    z = len(str(int(i)))
    s.add(z)
s

{1, 3, 4, 5}

In [176]:
final = []

def count_parkinglot(num, x, y):
  for k in range(10):
    if int(str(x)[0]) == k:
      if int(y[k]) == 0:
        if len(str(x))==3:
          final.append(17)
        elif len(str(x))==4:
          final.append(16)
        elif len(str(x))==5:
          final.append(20)
      else:
        n = int(x)//y[k]
        final.append(n)

for i in pp:
  if str(i) == "nan" or str(i) == "0":
    final.append(None)
  else:
    if len(str(int(i))) == 3:
      num=count_parkinglot(1, i, head2[1])
    elif len(str(int(i))) == 4:
      num=count_parkinglot(2, i, head2[2])
    elif len(str(int(i))) == 5:
      num=count_parkinglot(3, i, head2[3])
  count+=1


In [177]:
plan_parkinglot['주차면수']= final

plan_parkinglot['주차면수'].unique()
plan_parkinglot

,PRESENT_SN,DGM_NM,DGM_AR,DGM_LT,geometry,lon,lat,category,소유주체,주차면수
0,46230UQ152PS201202010003,기타 주차장시설,344,174,POINT (127.5842466468615 34.97703873009804),127.584247,34.977039,주차장,군유지,13.0
1,46230UQ152PS201202010002,기타 주차장시설,483,94,POINT (127.5838975833261 34.9783680381622),127.583898,34.978368,주차장,군유지,16.0
2,46230UQ152PS201202010109,노외주차장,1612,167,POINT (127.7513775866315 34.9457556003597),127.751378,34.945756,주차장,NaN,55.0
3,46230UQ152PS201202010106,노외주차장,305,70,POINT (127.7027049702007 34.98893978682275),127.702705,34.988940,주차장,군유지,11.0
4,46230UQ152PS201202010105,기타 주차장시설,4608,286,POINT (127.6896377942097 34.93744594755481),127.689638,34.937446,주차장,군유지,118.0
...,...,...,...,...,...,...,...,...,...,...
203,46230UQ152PS201202010153,노외주차장,805,127,POINT (127.5899412601061 34.97387051256062),127.589941,34.973871,주차장,개인,30.0
204,46230UQ152PS201202010143,기타 주차장시설,1244,144,POINT (127.6665902516217 34.93273694287271),127.666590,34.932737,주차장,NaN,42.0
205,46230UQ152PS201202010139,노외주차장,2657,208,POINT (127.5846080438416 34.97562896258442),127.584608,34.975629,주차장,군유지,88.0
206,46230UQ152PS201202010130,노외주차장,1313,142,POINT (127.6984953327121 34.92671326177527),127.698495,34.926713,주차장,군유지,45.0


In [181]:
plan_parkinglot.to_csv('plan_parkinglot.csv', index=False, encoding='cp949') #csv파일저장



---



---



---



---



---



In [ ]:
print(len(len_3), len(len_4), len(len_5))

84 120 3


In [ ]:
dic2= { "3":0, "4":0, "5":0}

n = sum(len_3)//len(len_3)
dic2['3']=n

n = sum(len_4)//len(len_4)
dic2['4']=n

n = sum(len_5)//len(len_5)
dic2['5']=n

dic2

{'3': 17.0, '4': 16.0, '5': 20.0}